In [ ]:
#-- Weghts = YOLO 11 --

In [ ]:
from IPython import display

In [ ]:
#-- Install ultralytics ------------------------------------------------------------------------------------------
!pip install ultralytics

display.clear_output()

import ultralytics
ultralytics.checks()
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Imports ----------------------------------------------------------------------------------------------------
from ultralytics import YOLO
from collections import defaultdict
import cv2
import numpy as np
import os
import time
import shutil
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Initialize --------------------------------------------------------------------------------------------------
out_dir = '/kaggle/working/'
weights_file = '/kaggle/input/yolo11-11frozen-13/model_11_frozen_epoch_60/train/weights/best.pt'

drone_files = ['/kaggle/input/drone-dataset-p1/v_5.mp4',
              '/kaggle/input/drone-dataset-p2/v_8.mp4',
              '/kaggle/input/drone-detection-test-videos-1/drone_video (1).mp4',
              '/kaggle/input/novin-data/Novin_Dataset/f2.part2.mp4',
              '/kaggle/input/sample-videos-detecting-and-matching-objs-1/sample_video_drone (5).mp4',
              '/kaggle/input/video-drone-bird-1/Untitled-13.mp4']

results_dir = out_dir + 'results/'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Set Detection Model ------------------------------------------------------------------------------------------
model = YOLO(weights_file)  
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Run ----------------------------------------------------------------------------------------------------------
time_dict = {}

for video_file in drone_files:
    
    start_time = time.time()
    
    #-- get video name --
    index = video_file.rfind('/')      
    video_name = video_file[index + 1:] 
    
    out_video_name = 'out_' + video_name    
    output_path = results_dir + out_video_name
    
    print(f'=== Processing {video_name} ================================')
    
    cap = cv2.VideoCapture(video_file)
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")  
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
    
    track_history = defaultdict(lambda: [])
    
    while cap.isOpened():    
        success, frame = cap.read()
        if success:
            results = model.track(frame,    
                                  tracker = 'bytetrack.yaml',
                                  persist=True,
                                  show = False)

            #-- Check if there are any detections --
            if results[0].boxes is not None and results[0].boxes.xywh is not None:            
                boxes = results[0].boxes.xywh.cpu()
                track_ids = results[0].boxes.id
                if track_ids is not None:
                    track_ids = track_ids.int().cpu().tolist()               
                    annotated_frame = results[0].plot()                
                    for box, track_id in zip(boxes, track_ids):
                        x, y, w, h = box
                        track = track_history[track_id]
                        track.append((float(x), float(y)))  #-- x, y center point
                        if len(track) > 30:  #-- retain 30 tracks
                            track.pop(0)

                        #-- Draw the tracking lines --
                        points = np.array(track, dtype=np.int32).reshape((-1, 1, 2))
                        cv2.polylines(annotated_frame, [points], isClosed=False, color=(0, 0, 255), thickness=4)
                else:
                    annotated_frame = frame  # If no IDs, use original frame

            else:
                annotated_frame = frame  # If no boxes, use original frame

            out.write(annotated_frame)

        else:
            # Break the loop if the end of the video is reached
            break

    #-- Release the video capture object and close the display window --
    cap.release()
    out.release()
    
    end_time = time.time()
    execution_time = end_time - start_time
    
    time_dict[video_name] = execution_time
    display.clear_output()    

In [ ]:
for video_name , time in time_dict.items():
    print(f'{video_name}: {time} s')

In [ ]:
zip_results = "results"
shutil.make_archive(zip_results, 'zip', results_dir)